# Import libraries and set the constans


In [1]:
#import pandas and other libraries for scrape, ETL
import pandas as pd
import numpy as np
import warnings
#cheat :)
import imdb
#pip install IMDbPY
import requests
from lxml import html
warnings.filterwarnings('ignore')
print('The moduls gets imported')


url = "https://www.imdb.com/chart/top/"

#Read the html table to top250[0]
top250_ls = pd.read_html(url) 

#From top250_ls[0] make a df
top250_df = top250_ls[0]

#Get the name and the rating
top250_ls = top250_df[top250_df.columns[1:3]]

#Get the top20 moovie
top20_df = top250_ls.head(20)
#Test
print(top20_df.head(20))


The moduls gets imported
                                    Rank & Title  IMDb Rating
0                    1.  A remény rabjai  (1994)          9.2
1                       2.  A keresztapa  (1972)          9.2
2                      3.  A sötét lovag  (2008)          9.0
3                    4.  A keresztapa II  (1974)          9.0
4               5.  Tizenkét dühös ember  (1957)          9.0
5                  6.  Schindler listája  (1993)          8.9
6   7.  A Gyűrűk Ura: A király visszatér  (2003)          8.9
7                       8.  Ponyvaregény  (1994)          8.9
8   9.  A Gyűrűk Ura: A gyűrű szövetsége  (2001)          8.8
9           10.  A Jó, a Rossz és a Csúf  (1966)          8.8
10                     11.  Forrest Gump  (1994)          8.8
11                  12.  Harcosok klubja  (1999)          8.8
12                           13.  Eredet  (2010)          8.7
13       14.  A Gyűrűk Ura: A két torony  (2002)          8.7
14            15.  A Birodalom visszavág  (19

# ETL

In [2]:
#Rename the columns
top20_df = top20_df.rename(columns={'Rank & Title': 'Name', 'IMDb Rating': 'Rating'})

#Get the name without year 
top20_df.Name = top20_df.Name.str.replace('\d+', '')
top20_df.Name = top20_df.Name.str.replace('(', '')
top20_df.Name = top20_df.Name.str.replace(')', '')
top20_df.Name = top20_df.Name.str.replace('.', '')
#Test
print(top20_df)

                                    Name  Rating
0                      A remény rabjai       9.2
1                         A keresztapa       9.2
2                        A sötét lovag       9.0
3                      A keresztapa II       9.0
4                 Tizenkét dühös ember       9.0
5                    Schindler listája       8.9
6     A Gyűrűk Ura: A király visszatér       8.9
7                         Ponyvaregény       8.9
8     A Gyűrűk Ura: A gyűrű szövetsége       8.8
9              A Jó, a Rossz és a Csúf       8.8
10                        Forrest Gump       8.8
11                     Harcosok klubja       8.8
12                              Eredet       8.7
13          A Gyűrűk Ura: A két torony       8.7
14               A Birodalom visszavág       8.7
15                              Mátrix       8.7
16                           Nagymenők       8.7
17             Száll a kakukk fészkére       8.6
18                             Hetedik       8.6
19                  

# Get the other data with id

In [3]:
#I use this library for searching the movie_id, then I make a url with constants
ia = imdb.IMDb()

#Make empty columns for the new data
top20_df["Id"] = " "
top20_df["Review"] = " "
top20_df["Ratings"] = " "
top20_df["Oscar"] = " "
  
#Iterate through the movies   
for i in range(len(top20_df)):
    #searching movie with name
    search = ia.search_movie(top20_df.iloc[i, 0])
    #Get the Movie_id
    top20_df.iloc[i, 2]=search[0].movieID
    #Make links for geting oscar, review and ratings
    url='https://www.imdb.com/title/tt'+top20_df.iloc[i,2]
    url_rev='https://www.imdb.com/title/tt'+top20_df.iloc[i,2]+'/reviews'
    url_rat='https://www.imdb.com/title/tt'+top20_df.iloc[i,2]+'/ratings/'
    #Make a request object 
    response=requests.get(url_rev)
    response2=requests.get(url_rat)
    response3=requests.get(url)
    #Formating the request
    tree = html.fromstring(response.content)
    tree2 = html.fromstring(response2.content)
    tree3 = html.fromstring(response3.content)
    #Fill the df wtih the data
    top20_df.iloc[i, 3]=tree.xpath('//div[@class="header"]/div/span/text()')
    top20_df.iloc[i, 4]=tree2.xpath('//section[@class="article listo"]/div/div[3]/div/text()')[0]
    top20_df.iloc[i, 5]=tree3.xpath('//ul[@class="ipc-metadata-list ipc-metadata-list--dividers-none sc-fcdc3619-2 kTHpcg ipc-metadata-list--base"]/li/a[1]/text()')
     

# ETL the numbers(Award,Review,Ratings)

In [4]:

top20_df['Review'] = top20_df['Review'].str.replace('Reviews','')
for i in range (len (top20_df)):
    if len(top20_df.iloc[i, 5].split())==3:
        top20_df.iloc[i, 5]=top20_df.iloc[i, 5].split()[1]
    else: 
        top20_df.iloc[i, 5] = 0
    top20_df.iloc[i, 4]=top20_df.iloc[i, 4].split()[0]
    
top20_df.head(20)
#Cast to int
top20_df.head(20)
top20_df.Review = top20_df.Review.str.replace(',', '')
top20_df.Ratings = top20_df.Ratings.str.replace(',', '')
top20_df.Review = top20_df.Review.astype(int)
top20_df.Ratings = top20_df.Ratings.astype(int)
top20_df.Oscar = top20_df.Oscar.astype(int)
print(top20_df.dtypes)

Name        object
Rating     float64
Id          object
Review       int32
Ratings      int32
Oscar        int32
dtype: object


# Review Penalizer:

In [5]:
#I think you meant here the Ratings  not review 
#auxiliary column
top20_df["Pan_Rating"] = " "
top20_df["Rating_adj"] = " "
#Get the max Ratings
maxrw = top20_df['Ratings'].max()
top20_df["Pan_Rating"] = top20_df['Ratings'].sub(maxrw)
top20_df['Pan_Rating'] = (top20_df['Pan_Rating'] / 100000)
top20_df['Pan_Rating']  = top20_df['Pan_Rating'].multiply(0.1)
top20_df["Rating_adj"] = top20_df['Rating'] + top20_df['Pan_Rating']
print(top20_df)


                                    Name  Rating       Id  Review  Ratings  \
0                      A remény rabjai       9.2  0111161   10092  2572038   
1                         A keresztapa       9.2  0068646    4933  1771170   
2                        A sötét lovag       9.0  0468569    8067  2538237   
3                      A keresztapa II       9.0  0071562    1273  1224839   
4                 Tizenkét dühös ember       9.0  0050083    1954   759599   
5                    Schindler listája       8.9  0108052    2098  1310243   
6     A Gyűrűk Ura: A király visszatér       8.9  0167260    4023  1769217   
7                         Ponyvaregény       8.9  0110912    3459  1974228   
8     A Gyűrűk Ura: A gyűrű szövetsége       8.8  0120737    5622  1790853   
9              A Jó, a Rossz és a Csúf       8.8  0060196    1276   739468   
10                        Forrest Gump       8.8  0109830    2907  1984047   
11                     Harcosok klubja       8.8  0137523    425

# Oscar Calculator

In [6]:

top20_df["Award_Rat"] = " "
for a  in range (len (top20_df)):
    if top20_df.iloc[a, 5] >= 1 and top20_df.iloc[a, 5]<=2: 
        top20_df["Award_Rat"][a]=0.3 
        continue
    elif top20_df.iloc[a, 5] >= 3 and top20_df.iloc[a, 5]<=5:
        top20_df["Award_Rat"][a]=0.5 
        continue
    elif top20_df.iloc[a, 5]>= 6 and top20_df.iloc[a, 5]<=10:
        top20_df["Award_Rat"][a]=1 
        continue
    elif top20_df.iloc[a, 5] >10:
        top20_df["Award_Rat"][a]=1.5
    else:
        top20_df["Award_Rat"][a]=0
top20_df["Rating_adj"] = top20_df['Rating_adj'] + top20_df['Award_Rat']
print(top20_df)

                                    Name  Rating       Id  Review  Ratings  \
0                      A remény rabjai       9.2  0111161   10092  2572038   
1                         A keresztapa       9.2  0068646    4933  1771170   
2                        A sötét lovag       9.0  0468569    8067  2538237   
3                      A keresztapa II       9.0  0071562    1273  1224839   
4                 Tizenkét dühös ember       9.0  0050083    1954   759599   
5                    Schindler listája       8.9  0108052    2098  1310243   
6     A Gyűrűk Ura: A király visszatér       8.9  0167260    4023  1769217   
7                         Ponyvaregény       8.9  0110912    3459  1974228   
8     A Gyűrűk Ura: A gyűrű szövetsége       8.8  0120737    5622  1790853   
9              A Jó, a Rossz és a Csúf       8.8  0060196    1276   739468   
10                        Forrest Gump       8.8  0109830    2907  1984047   
11                     Harcosok klubja       8.8  0137523    425

# Writeout the file


In [7]:
#Delete the useless columns
top20_df = top20_df.drop('Award_Rat', 1)
top20_df = top20_df.drop('Pan_Rating', 1)
top20_df = top20_df.drop('Id', 1)
#descending
top20_df.sort_values(by = ['Rating', 'Rating_adj'], ascending = [False, False])
#Writeout the file
top20_df.to_csv('top20_moovie_f.txt', encoding='utf-8')
top20_df.head(200)

,Name,Rating,Review,Ratings,Oscar,Rating_adj
0,A remény rabjai,9.2,10092,2572038,0,9.2
1,A keresztapa,9.2,4933,1771170,3,8.899132
2,A sötét lovag,9.0,8067,2538237,2,9.266199
3,A keresztapa II,9.0,1273,1224839,6,8.652801
4,Tizenkét dühös ember,9.0,1954,759599,0,7.187561
5,Schindler listája,8.9,2098,1310243,7,8.638205
6,A Gyűrűk Ura: A király visszatér,8.9,4023,1769217,11,9.597179
7,Ponyvaregény,8.9,3459,1974228,1,8.60219
8,A Gyűrűk Ura: A gyűrű szövetsége,8.8,5622,1790853,4,8.518815
9,"A Jó, a Rossz és a Csúf",8.8,1276,739468,0,6.96743
